<h1>The apartment finder:</h1>

<h2>First, we ge the apartment information from our Kijiji scraper:</h2>

In [90]:
                                                                                                                                                                                                    #Get informations from Kijiji scraper:
%run kijiji_scraper.ipynb

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/home/kurosuzume/.wdm/drivers/chromedriver/linux64/86.0.4240.22/chromedriver] found in cache


 
There are no more apartments with those filters right now. 
chrome closed


In [91]:
import html
from IPython.display import HTML


<h2>The apartment addresses are shown below.</h2>

In [93]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h2>Now, we import all libraries needed:</h2>

In [94]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files

!pip install pgeocode
import pgeocode

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't competed 
from geopy.geocoders import Nominatim

#Matplotlib and associated ploting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium-0.5.0 --yes
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from geopy import distance  
print("Libraries imported successfully")



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - folium-0.5.0

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported successfully


<h2>The reference point is the NBCC campus in New Brunswick.</h2>
<h3>We set the location coordinates and the radius we will use to find the apartments and amenities:</h3>

In [95]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="NB_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000


The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Ensuring all te data is in the right format:</h3>

In [97]:
#Before finding the geo-locations of the addresses given, we need to verify if they are in the right format:
main_address = []
unaccepted_address_index = []

for address in text_address:
    house = address.split(',')
    house = house[0]+",Fredericton, CA"
    main_address.append(house)

for i in range(len(main_address)):
    if main_address is None:    
        main_address.remove(main_address[i])
        ap_prices.remove(ap_prices[i])
        title_pic_change.remove(title_pic_change[i])
        num_bedrooms.remove(num_bedrooms[i])
        num_bathrooms.remove(num_bathrooms[i])

print("Data format verified!")

Data format verified!


In [98]:
geo_location = main_address

<h2>Finding the geographic location of each apartment:</h2>

In [99]:
#Now, to find the geo location of each of the addresses:

index_to_remove = []
geo_location = []
for address in range(len(main_address)):
    appended_location = geolocator.geocode(main_address[address])
    geo_location.append(appended_location)
    if appended_location is None:
        index_to_remove.append(address)
index_to_remove = index_to_remove.reverse()

if index_to_remove is None:
        pass
else: 
    for i in index_to_remove:

        geo_location.remove(geo_location[address])
        ap_prices.remove(ap_prices[address])
        num_bedrooms.remove(num_bedrooms[address])
        num_bathrooms.remove(num_bathrooms[address])          

print("geo location acquired.")

geo location acquired.


<h3>Showing the Location information for all the apartments</h3>

In [100]:
geo_location

[None,
 Location(Irvine Street, Neill Brae, Youngs Crossing, Fredericton, York County, New Brunswick, E3A 9M1, Canada, (45.9715686, -66.6137731, 0.0)),
 Location(71, Angelview Court, Morning Gate, Fredericton, York County, New Brunswick, E3B 5Z1, Canada, (45.96456025, -66.6929489371789, 0.0)),
 None,
 Location(Murray Avenue, Neill Brae, Cliffe Crossing, Fredericton, York County, New Brunswick, E3A 9M1, Canada, (45.972481, -66.6127245, 0.0)),
 Location(Union Street, North Devon, Sitansisk (St. Mary's First Nation), Fredericton, York County, New Brunswick, E3A 3M8, Canada, (45.9712213, -66.6349744, 0.0)),
 Location(Wetmore Road, Dunn's Crossing, Southwood Park, Fredericton, York County, New Brunswick, E3B 5H1, Canada, (45.9236635, -66.6321001, 0.0)),
 Location(Wellington Court, Sandyville, Fredericton, York County, New Brunswick, E3A 4A3, Canada, (45.997895, -66.5781018, 0.0)),
 Location(Brookside Drive, Royal Road, Nashwaaksis, Fredericton, York County, New Brunswick, E3A 1B8, Canada, (

In [101]:
location_info = []
discarded_index = []
for location in range(len(geo_location)):
    if not geo_location[location] == None: 
        location_info.append([geo_location[location].address, geo_location[location].latitude, geo_location[location].longitude ])
    else: 
        discarded_index.append(location)
    

<h3>Using the Introduction_to_the_Problem notebook to retrieve the map.</h3>

In [103]:
#Get the map code from the introduction notebook:
%run Introduction_to_the_Problem.ipynb

The college address is: Duffie Drive, College Hill, Skyline Acres, Fredericton, York County, New Brunswick, E3B 9W4, Canada


In [104]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000



The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Foursquare credentials:</h3>
<p>Please, fill with your foursquare information:</p>

In [105]:
CLIENT_ID = 'GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4' # your Foursquare ID
CLIENT_SECRET = 'L354OYVY1EG0MKKQCKGD30RQ2ZE2KPAIJDKB5EPTJ4RW3DGT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID + 'PLEASE VERIFY!')
print('CLIENT_SECRET:' + CLIENT_SECRET + 'PLEASE VERIFY!')

Your credentails:
CLIENT_ID: GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4PLEASE VERIFY!
CLIENT_SECRET:L354OYVY1EG0MKKQCKGD30RQ2ZE2KPAIJDKB5EPTJ4RW3DGTPLEASE VERIFY!


<p>Since it wasn't possible to use an automatic iterative method to find the amenities, we will do it manually. for that we will use the Foursquare API and acquire nearby venues data:</p>

In [106]:
#Now, to get the nearby venues according to the important amenities:
amenities_list = ["Grocery Store","Supermarket", "Gym", "Drugstore", "Hospital", "Candy Store"]
search_query0 = "Grocery Store"
search_query1 = "Supermarket"
search_query2 = "Gym"
search_query3 = "Drugstore"
search_query4 = "Hospital"
search_query5 = "Candy Store"


In [107]:
url0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query0, radius, LIMIT)
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query4, radius, LIMIT)
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query5, radius, LIMIT)


In [108]:
results0 = requests.get(url0).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()
results5 = requests.get(url5).json()



In [109]:
# assign relevant part of JSON to venues
venues0 = results0['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']
venues5 = results5['response']['venues']

# transform venues into dataframes
dataframe0 = pd.json_normalize(venues0)
dataframe1 = pd.json_normalize(venues1)
dataframe2 = pd.json_normalize(venues2)
dataframe3 = pd.json_normalize(venues3)
dataframe4 = pd.json_normalize(venues4)
dataframe5 = pd.json_normalize(venues5)


In [110]:
print("Venues Data retrieved.")

Venues Data retrieved.


<h2>Results from foursquare will converge into one dataframe:</h2>

In [1]:
# now converge all results into one dataframe:

dataframe0 = pd.concat([dataframe0,dataframe1], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe2], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe3], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe4], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe5], axis=0, ignore_index=True)



dataframe0.head

NameError: name 'pd' is not defined

In [112]:
dataframe0.shape

(96, 18)

<h2>Filtering the pandas dataframe:</h2>

In [113]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe0.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe0.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Dick's Grocery Store,Smoke Shop,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,4c545e5db426ef3b11cc7e8a
1,Dollar Store,Miscellaneous Shop,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4badfab8f964a520a3773be3
2,Great Canadian Dollar Store,Discount Store,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,53f7c437498eacb60d222490
3,American Eagle & Aerie Store,Clothing Store,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,4c3c8e25282203bb55a3fcda
4,The UPS Store,Miscellaneous Shop,527 Dundonald St,45.954230,-66.647487,"[{'label': 'display', 'lat': 45.95422952664039...",1271,E3B 1X5,CA,Fredericton,NB,Canada,"[527 Dundonald St, Fredericton NB E3B 1X5, Can...",NaN,5b93ade0f193c0002c473a51
5,UPS Store,Paper / Office Supplies Store,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,4c1030d7ce57c928688981d2
6,Walmart Grocery Pickup,Supermarket,1399 Regent St,45.934115,-66.664224,"[{'label': 'display', 'lat': 45.93411510382234...",1740,E3C 1A3,CA,Fredericton,NB,Canada,"[1399 Regent St, Fredericton NB E3C 1A3, Canada]",NaN,5f81ce3ec0146e296bc5f5cb
7,Kwik-Way Convenience Store,Convenience Store,NaN,45.955823,-66.652087,"[{'label': 'display', 'lat': 45.95582316108977...",1525,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4f0a2271e4b0e8ed394e6f81
8,Everything For A Dollar Store,Miscellaneous Shop,NaN,45.940661,-66.664114,"[{'label': 'display', 'lat': 45.94066140773827...",1453,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4cd979481647a0936c21e74c
9,BellAliant Store,Mobile Phone Shop,NaN,45.934620,-66.663767,"[{'label': 'display', 'lat': 45.93462, 'lng': ...",1679,NaN,CA,NaN,NaN,Canada,[Canada],NaN,51ba06ad498e1994a7616f39


In [114]:
amenities= dataframe_filtered["name"].to_list()
amenities_categories = dataframe_filtered["categories"].to_list()
amenities_list_lat = dataframe_filtered["lat"].to_list()
amenities_list_lng = dataframe_filtered["lng"].to_list()


<h3>And finally, we add the amenities to our Folium map.</h3>
<p>The amenities will be displayed as a blue dot with brown outline. </p>

In [115]:

for i in range(len(geo_location)):
    if not geo_location[i] == None: 
        tooltip = "Apartment"
        folium.Marker([geo_location[i].latitude,geo_location[i].longitude], popup=AD_title[i], tooltip = tooltip ).add_to(map_Fred)
    else:
        pass

In [116]:

# add the Italian amenities as blue circle markers with red outline
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip = label
    ).add_to(map_Fred)
map_Fred

<h3>Now, using geopy, we can verify amenities within a 800m radius from an apartment.</h3>

In [117]:
from geopy import point
#Function to find amenities in a 500m radius from a center point:
def find_in_radius(center_name, center_lat,center_lng, amenities_list_lat, amenities_list_lng, amenities_list, amenities_categories):
    """(float, float, list) -> list
        Retruns a list of amenities within a 800 radius from a center point.
    """
    center_lat = str(center_lat)
    center_lng = str(center_lng)
    center_point = point.Point(str(center_lat)+ "," + str(center_lng))
    
    within_radius = []
    amenities_cat = []
    for amenity in range(len(amenities_list)):
        amenities_list_point = point.Point(str(amenities_list_lat[amenity])+ "," + str(amenities_list_lng[amenity]))
        amen_dis = distance.distance(center_point, amenities_list_point).km
        if amen_dis <= 0.80: #1000 meters
            within_radius.append(amenities_list[amenity])
            amenities_cat.append(amenities_categories[amenity])
    return center_name,within_radius,amenities_cat

        

In [118]:
for type in range(len(amenities_categories)):
    if amenities_categories[type] == None:
        amenities_categories[type] = "None"

In [119]:
full_list = []
for i in range(len(location_info)):
    full_list.append(find_in_radius(location_info[i][0],location_info[i][1], location_info[i][2], amenities_list_lat, amenities_list_lng, amenities,amenities_categories))

In [121]:
reverse_discarded_index = discarded_index.reverse()

In [124]:
for i in discarded_index:
    ap_prices.remove(ap_prices[i])
    num_bedrooms.remove(num_bedrooms[i])
    num_bathrooms.remove(num_bathrooms[i])

In [126]:
ap_prices_float = []
for i in range(len(ap_prices)):
    split = ap_prices[i].replace('$', '')
    
    ap_prices_float.append(split)
    

In [127]:
ap_prices_final = []
for i in range(len(ap_prices_float)):
    make_float = float(ap_prices_float[i].replace(',',''))
    ap_prices_final.append(make_float)
ap_prices = ap_prices_final

In [128]:
apt_address = []
amenity_name = []
amenity_type= []
for i in range(len(full_list)):
    apt_address.append(full_list[i][0])
    amenity_name.append(full_list[i][1])
    amenity_type.append(full_list[i][2])

In [129]:
separator = ","
prepared_amenity = []
for i in range(len(amenity_type)):
    sep = separator.join(amenity_type[i])
    prepared_amenity.append(sep)


In [130]:
for i in range(len(prepared_amenity)):
    prepared_amenity[i] = str(prepared_amenity[i]).split(',')
    for j in range(len(prepared_amenity[i])):
        prepared_amenity[i][j].split(",")

    

In [132]:
apartment_df= pd.DataFrame(list(zip(apt_address, amenity_name, prepared_amenity, ap_prices, num_bedrooms, num_bathrooms)))

In [133]:
len(prepared_amenity)
result = list(set(x for l in prepared_amenity for x in l))


In [135]:
apartment_df = pd.concat([apartment_df,pd.get_dummies(apartment_df[2].apply(pd.Series).stack()).sum(level=0)], axis = 1)

In [136]:
apartment_df = apartment_df.replace({5:{'Bathrooms: 1': 1, 'Bathrooms: 2': 2, "Bathrooms: 1.5": 2}})

apartment_df = apartment_df.replace({4:{'Bedrooms: Bachelor/Studio': 0, 'Bedrooms: 1': 1,'Bedrooms: 2': 2, 'Bedrooms: 1 + Den':2}})


In [137]:
apartment_df = apartment_df.rename(columns={0: "Apartment Address", 1: "Amenities",2:"Amenity type", 3:"Rent price", 4:"Bedrooms", 5:"Bathrooms"})


In [138]:
from datetime import date
date.fromisoformat('2019-12-04')


datetime.date(2019, 12, 4)

#print(discarded_index)
#print(len(ap_prices))<h3>with all that information, we're able to create the apartment DataFrame:</h3>

In [141]:
for i in range(len(apartment_df)):
    apartment_df['date'] = date.today()
    

In [142]:
apartment_df.head(1)

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Nudist Beach,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date
0,"Irvine Street, Neill Brae, Youngs Crossing, Fr...",[],[],595.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-11-01


In [143]:
apartment_df.shape

(31, 28)

In [144]:
apartment_df[''] = apartment_df.rename(columns={'':'empty'})

<h3>Then, we open the csv file with an older instance of the DF, and append the unique new values between them</h3>

In [145]:
apartment_df_initial = pd.read_csv("apartment_dataframe.csv", header = 0, index_col = False)


In [146]:
apartment_df_initial = apartment_df_initial.rename(columns={'Unnamed: 6':'empty'})

In [147]:
apartment_df_initial

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0,0.0,2,2,0,0,0,0.0,2,0,0,0,2,0,1,2020-10-25
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0,2.0,0,0,0,0,0,0.0,2,0,0,2,0,0,1,2020-10-25
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,2,0,1,2,0,0,1,2020-10-25
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,0,0,1,0,0,0,0,2020-10-25
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25


In [148]:
apartment_df_initial.shape

(59, 26)

In [149]:
apartment_df_concat = pd.concat([apartment_df_initial,apartment_df]).reset_index(drop=True)

In [150]:
apartment_df_concat.shape

(90, 29)

In [151]:
apartment_df_concat

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date,,Grocery Store,Nudist Beach
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0,0.0,2,2,0,0,0,0.0,2,0,0,0,2,0,1,2020-10-25,NaN,NaN,NaN
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25,NaN,NaN,NaN
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25,NaN,NaN,NaN
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0,2.0,0,0,0,0,0,0.0,2,0,0,2,0,0,1,2020-10-25,NaN,NaN,NaN
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,2,0,1,2,0,0,1,2020-10-25,NaN,NaN,NaN
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,0,0,1,0,0,0,0,2020-10-25,NaN,NaN,NaN
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN


In [152]:
#apartment_df_concat = apartment_df_concat.drop(columns=['', 'Nudist Beach'])

In [153]:
apartment_df = apartment_df_concat.drop_duplicates(keep='first', subset=['Apartment Address', 'Rent price'])

In [154]:
#apartment_df = apartment_df.drop(['Unnamed: 0'], axis=1)

In [155]:
apartment_df

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,empty,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Furniture / Home Store,General Travel,Gym,Gym / Fitness Center,Hardware Store,Hospital,Miscellaneous Shop,Paper / Office Supplies Store,Smoke Shop,Thrift / Vintage Store,Tourist Information Center,Veterinarian,date,,Grocery Store,Nudist Beach
0,"Priestman Street, Rabbit Town, Uptown, Frederi...","[""Dick's Grocery Store"", 'Great Canadian Dolla...","['Smoke Shop', 'Discount Store', 'Department S...",450.0,0,1,0.0,0,0.0,0.0,0,0.0,2,2,0,0,0,0.0,2,0,0,0,2,0,1,2020-10-25,NaN,NaN,NaN
1,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",500.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25,NaN,NaN,NaN
2,"George Street, Carleton, Downtown, Fredericton...","['Urban Almanac', 'UPS Store', 'The UPS Store'...","['Furniture / Home Store', 'Paper / Office Sup...",550.0,0,1,0.0,0,2.0,0.0,0,2.0,0,0,2,2,1,0.0,2,0,1,2,0,2,1,2020-10-25,NaN,NaN,NaN
3,"York Street, Rabbit Town, Downtown, Fredericto...","['UPS Store', 'The UPS Store', 'Kwik-Way Conve...","['Paper / Office Supplies Store', 'Miscellaneo...",600.0,0,1,0.0,0,0.0,0.0,0,2.0,0,0,0,0,0,0.0,2,0,0,2,0,0,1,2020-10-25,NaN,NaN,NaN
4,"Saint John Street, Queen's Park, Downtown, Fre...","['UPS Store', 'University Cafe & Market Conven...","['Paper / Office Supplies Store', 'Candy Store...",600.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,2,0,1,2,0,0,1,2020-10-25,NaN,NaN,NaN
5,"31, Corbett Avenue, Station Pointe, Barker's P...",[],[''],700.0,0,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
6,"36-38, Waterloo Row, Town Platt, Downtown, Fre...","['University Cafe & Market Convenience Store',...","['Candy Store', 'General Travel', 'Clothing St...",748.0,0,1,0.0,2,2.0,0.0,0,0.0,0,0,0,2,1,0.0,0,0,1,0,0,0,0,2020-10-25,NaN,NaN,NaN
7,"706, Valleyview Street, College Hill, Skyline ...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
8,"Valleyview Street, College Hill, Skyline Acres...","['Dollar Store', 'Lady Beaverbrook Gym - UNB',...","['Miscellaneous Shop', 'Gym / Fitness Center',...",750.0,1,1,0.0,0,0.0,2.0,1,0.0,0,0,0,0,1,0.0,2,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN
9,"Woodstock Road, Garden Creek, Fredericton, Yor...",[],[''],775.0,1,1,1.0,0,0.0,0.0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,2020-10-25,NaN,NaN,NaN


In [156]:
apartment_df.to_csv("apartment_dataframe.csv",mode='w', header=True, index=False)

print("The apartment_dataframe.csv has been updated!")

The apartment_dataframe.csv has been updated!


<p>Now, we will use this information to analyse the results and generate a conclusion.</p>